In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/friedman/many"
results_dir_tanh = "results/regression/single_layer/tanh/friedman_correlated"

model_names_tanh_corr = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]
model_names_tanh_nodewise_corr = ["Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise", "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]

tanh_fits_corr = {}
tanh_fits_nodewise_corr = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    
    tanh_fit_corr = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_corr,
        include_prior=False,
    )
    tanh_fit_nodewise_corr = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_nodewise_corr,
        include_prior=False,
    )

    tanh_fits_corr[base_config_name] = tanh_fit_corr  # use clean key
    tanh_fits_nodewise_corr[base_config_name] = tanh_fit_nodewise_corr  # use clean key

In [ ]:
data_dir = f"datasets/friedman"
results_dir_tanh = "results/regression/single_layer/tanh/friedman_correlated"

model_names_tanh_corr = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]
model_names_tanh_nodewise_corr = ["Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise", "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    
    tanh_fit_corr = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_corr,
        include_prior=False,
    )
    tanh_fit_nodewise_corr = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_nodewise_corr,
        include_prior=False,
    )

    tanh_fits_corr[base_config_name] = tanh_fit_corr  # use clean key
    tanh_fits_nodewise_corr[base_config_name] = tanh_fit_nodewise_corr  # use clean key

In [4]:
import re
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

def _find_run_key(fits_by_run, *, N, D, sigma, seed, correlated=False):
    """
    Tries to find the right key inside e.g. tanh_fits for the requested (N, D, sigma, seed).
    Works even if your sigma formatting differs a bit (e.g. 1, 1.0, 1.00).
    """
    # Common naming you showed: Friedman_N100_p10_sigma1.00_seed3
    # If you also have "CorrelatedFriedman" etc, we try both patterns.
    sigma_strs = [
        f"{sigma}",
        f"{sigma:.1f}",
        f"{sigma:.2f}",
        f"{sigma:.3f}",
    ]
    candidates = []
    for s in sigma_strs:
        if correlated:
            candidates.append(f"CorrelatedFriedman_N{N}_p{D}_sigma{s}_seed{seed}")
        candidates.append(f"Friedman_N{N}_p{D}_sigma{s}_seed{seed}")

    for k in candidates:
        if k in fits_by_run:
            return k

    # If exact match not found, do a regex search (robust to extra prefixes/suffixes)
    # Example matches: "...Friedman_N100_p10_sigma1.00_seed3..."
    sig_pat = "|".join(re.escape(s) for s in sigma_strs)
    base = r"Friedman" if not correlated else r"(?:CorrelatedFriedman|Friedman)"
    pat = re.compile(
        rf"{base}_N{N}_p{D}_sigma(?:{sig_pat})_seed{seed}"
    )
    matches = [k for k in fits_by_run.keys() if pat.search(k)]
    if len(matches) == 1:
        return matches[0]
    if len(matches) > 1:
        # Prefer the shortest/most exact-looking key
        matches = sorted(matches, key=len)
        return matches[0]

    raise KeyError(
        f"Could not find a run in fits_by_run for N={N}, D={D}, sigma={sigma}, seed={seed}, correlated={correlated}.\n"
        f"Example expected key: 'Friedman_N{N}_p{D}_sigma{sigma:.2f}_seed{seed}'."
    )

def build_global_mask_from_posterior(
    W_samples,
    sparsity,
    method="Eabs",          # "Eabs" or "Eabs_stability"
    stability_quantile=0.1, # used if method="Eabs_stability"
    prune_smallest=True
):
    """
    W_samples: array (S, ..., ...) posterior draws of a weight matrix.
    sparsity: fraction to prune (q). Keeps (1-q).
    Returns mask with same trailing shape as one draw, dtype float {0,1}.
    """
    assert 0.0 <= sparsity < 1.0
    S = W_samples.shape[0]
    W_abs = np.abs(W_samples)  # (S, ...)

    # Importance score a = E|w|
    a = W_abs.mean(axis=0)     # (..., ...)

    if method == "Eabs":
        score = a
    elif method == "Eabs_stability":
        # Stability proxy pi = P(|w| > t), where t is a small global quantile of |w|
        t = np.quantile(W_abs.reshape(S, -1), stability_quantile)
        pi = (W_abs > t).mean(axis=0)
        # Combine: emphasize both "large on average" and "consistently non-tiny"
        score = a * pi
    else:
        raise ValueError("method must be 'Eabs' or 'Eabs_stability'")

    # Decide how many to prune
    num_params = score.size
    k_prune = int(np.floor(sparsity * num_params))
    if k_prune == 0:
        return np.ones_like(score, dtype=float)

    flat = score.reshape(-1)

    if prune_smallest:
        # prune lowest scores
        thresh = np.partition(flat, k_prune - 1)[k_prune - 1]
        mask = (score > thresh).astype(float)
        # if ties create too many kept/pruned, fix deterministically
        # (rare but possible with many equal scores)
        if mask.sum() > num_params - k_prune:
            # drop some tied-at-threshold entries
            idx_tied = np.where(score.reshape(-1) == thresh)[0]
            need_drop = int(mask.sum() - (num_params - k_prune))
            if need_drop > 0:
                mask_flat = mask.reshape(-1)
                mask_flat[idx_tied[:need_drop]] = 0.0
                mask = mask_flat.reshape(score.shape)
        elif mask.sum() < num_params - k_prune:
            # add some tied entries if we kept too few
            idx_tied = np.where(score.reshape(-1) == thresh)[0]
            need_add = int((num_params - k_prune) - mask.sum())
            if need_add > 0:
                mask_flat = mask.reshape(-1)
                # add back from tied
                add_candidates = idx_tied[mask_flat[idx_tied] == 0.0]
                mask_flat[add_candidates[:need_add]] = 1.0
                mask = mask_flat.reshape(score.shape)
    else:
        # prune largest (not typical)
        thresh = np.partition(flat, num_params - k_prune)[num_params - k_prune]
        mask = (score < thresh).astype(float)

    return mask


def precompute_global_masks(
    all_fits,
    dataset_key,
    model,
    sparsity_levels,
    prune_W2=False,
    method="Eabs_stability"
):
    """
    Returns dict: sparsity -> (mask_W1, mask_W2 or None)
    """
    fit = all_fits[dataset_key][model]["posterior"]

    W1_samples = fit.stan_variable("W_1")  # (S, P, H)
    W2_samples = fit.stan_variable("W_L")  # (S, H, O) or (S, H) depending on O

    masks = {}
    for q in sparsity_levels:
        mask_W1 = build_global_mask_from_posterior(W1_samples, q, method=method)
        mask_W2 = None
        if prune_W2:
            mask_W2 = build_global_mask_from_posterior(W2_samples, q, method=method)
        masks[q] = (mask_W1, mask_W2)
    return masks


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from utils.generate_data import sample_gaussian_copula_uniform

def generate_Friedman_data_v2(N=100, D=10, sigma=1.0, test_size=0.2, seed=42, standardize_y=True, return_scale=True):
    np.random.seed(seed)
    X = np.random.uniform(0, 1, size=(N, D))
    x0, x1, x2, x3, x4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5.0 * x4
    )
    y = y_clean + np.random.normal(0, sigma, size=N)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    if not standardize_y:
        return (X_train, X_test, y_train, y_test) if not return_scale else (X_train, X_test, y_train, y_test, 0.0, 1.0)

    y_mean = y_train.mean()
    y_std = y_train.std() if y_train.std() > 0 else 1.0

    y_train_s = (y_train - y_mean) / y_std
    y_test_s = (y_test - y_mean) / y_std

    if return_scale:
        return X_train, X_test, y_train_s, y_test_s, y_mean, y_std
    return X_train, X_test, y_train_s, y_test_s

def generate_correlated_Friedman_data_v2(N=100, D=10, sigma=1.0, test_size=0.2, seed=42, standardize_y=True, return_scale=True):
    """
    Generate synthetic regression data for Bayesian neural network experiments.

    Parameters:
        N (int): Number of samples.
        D (int): Number of features.
        sigma (float): Noise level.
        test_size (float): Proportion for test split.
        seed (int): Random seed.
        standardize_y (bool): Whether to standardize the response variable.

    Returns:
        tuple: (X_train, X_test, y_train, y_test, y_mean, y_std) if standardize_y,
               else (X_train, X_test, y_train, y_test)
    """
    np.random.seed(seed)
    d = 10
    S_custom = np.eye(d)
    # Block 1 (vars 0..4): high Spearman, 0.7
    for i in range(0, 3):
        for j in range(i+1, 3):
            S_custom[i, j] = S_custom[j, i] = 0.8
    # Block 2 (vars 5..9): moderate Spearman, 0.4
    for i in range(5, 10):
        for j in range(i+1, 10):
            S_custom[i, j] = S_custom[j, i] = -0.5
    # Cross-block weaker, 0.15
    for i in range(0, 5):
        for j in range(5, 10):
            S_custom[i, j] = S_custom[j, i] = 0.15
    # A couple of bespoke pairs:
    S_custom[0, 9] = S_custom[9, 0] = 0.4
    S_custom[2, 7] = S_custom[7, 2] = 0.9  # very strong (will be projected if infeasible)
    S_custom[3, 4] = S_custom[4, 3] = -0.9  # very strong (will be projected if infeasible)
    S_custom[1, 6] = S_custom[6, 1] = -0.9  # very strong (will be projected if infeasible)

    U, _ = sample_gaussian_copula_uniform(n=10000, S=S_custom, random_state=123)
    #X = np.random.uniform(0, 1, size=(N, D))
    if N != U.shape[0]:
        idx = np.random.choice(U.shape[0], size=N, replace=False)
        X = U[idx, :]
    else:
        X = U

    x0, x1, x2, x3, x4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5.0 * x4
    )

    y = y_clean + np.random.normal(0, sigma, size=N)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    if not standardize_y:
        return (X_train, X_test, y_train, y_test) if not return_scale else (X_train, X_test, y_train, y_test, 0.0, 1.0)

    y_mean = y_train.mean()
    y_std = y_train.std() if y_train.std() > 0 else 1.0

    y_train_s = (y_train - y_mean) / y_std
    y_test_s = (y_test - y_mean) / y_std

    if return_scale:
        return X_train, X_test, y_train_s, y_test_s, y_mean, y_std
    return X_train, X_test, y_train_s, y_test_s

def make_large_eval_set(
    generator_fn,
    N_train,
    D,
    sigma,
    seed,
    n_eval=5000,
    standardize_y=True
):
    """
    Returns X_eval, y_eval (standardized if standardize_y=True), plus y_mean,y_std
    defined from the training split.
    """
    N_total = N_train + n_eval

    X_tr, X_te, y_tr, y_te, y_mean, y_std = generator_fn(
        N=N_total, D=D, sigma=sigma, test_size=n_eval / N_total, seed=seed,
        standardize_y=standardize_y, return_scale=True
    )
    # Now X_te has approx n_eval points (exact given test_size construction).
    return X_te, np.asarray(y_te).squeeze(), y_mean, y_std


def _logsumexp(a, axis=None):
    amax = np.max(a, axis=axis, keepdims=True)
    out = amax + np.log(np.sum(np.exp(a - amax), axis=axis, keepdims=True))
    return np.squeeze(out, axis=axis)

def gaussian_nll_pointwise(y, mu, sigma):
    return 0.5*np.log(2*np.pi*(sigma**2)) + 0.5*((y-mu)**2)/(sigma**2)

def compute_sparse_metrics_results_globalmask_large_eval(
    seeds, all_seeds, models, all_fits, get_N_sigma, forward_pass,
    folder,
    sparsity=0.0,
    masks_cache=None,
    prune_W2=False,
    compute_nll=True,
    noise_var_name="sigma",
    n_eval=5000,
    D=10,
    standardize_y=True,
    # pass the correct generator functions
    gen_uncorr=None,
    gen_corr=None,
):
    """
    Evaluate on a large generated test set instead of the stored tiny X_test/y_test.
    Assumes model was trained on standardized y if standardize_y=True.
    """
    assert gen_uncorr is not None and gen_corr is not None, "Pass both generator functions."

    results = []
    posterior_means = []

    # choose generator based on folder name
    def choose_gen(folder):
        return gen_corr if "friedman_correlated" in folder else gen_uncorr

    for seed in seeds:
        N, sigma = get_N_sigma(seed, all_seeds)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'

        # Build large eval set consistent with training split standardization
        gen_fn = choose_gen(folder)
        X_test, y_test, y_mean, y_std = make_large_eval_set(
            generator_fn=gen_fn,
            N_train=N,
            D=D,
            sigma=sigma,
            seed=seed,
            n_eval=n_eval,
            standardize_y=standardize_y
        )

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                W1_samples = fit.stan_variable("W_1")           # (S, P, H)
                W2_samples = fit.stan_variable("W_L")           # (S, H, O)
                b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
                b2_samples = fit.stan_variable("output_bias")   # (S, O)

                noise_samples = None
                if compute_nll:
                    try:
                        noise_samples = fit.stan_variable(noise_var_name).squeeze()
                    except Exception:
                        noise_samples = None
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = W1_samples.shape[0]
            y_hats = np.zeros((S, y_test.shape[0]))
            #rmses = np.zeros(S)

            mask_W1 = mask_W2 = None
            if masks_cache is not None and sparsity > 0.0:
                mask_W1, mask_W2 = masks_cache[(dataset_key, model)][sparsity]

            for i in range(S):
                W1 = W1_samples[i]
                W2 = W2_samples[i]

                if mask_W1 is not None:
                    W1 = W1 * mask_W1
                if prune_W2 and (mask_W2 is not None):
                    W2 = W2 * mask_W2

                y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i]).squeeze()
                y_hats[i] = y_hat
                #rmses[i] = np.sqrt(np.mean((y_hat - y_test)**2))

            # posterior mean RMSE (standardized scale)
            posterior_mean = y_hats.mean(axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test)**2))

            out_pm = {
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'sparsity': sparsity,
                'n_eval': y_test.shape[0],
                'posterior_mean_rmse': posterior_mean_rmse,
                'posterior_mean_rmse_orig': posterior_mean_rmse * y_std,  # back to original y scale
            }

            if compute_nll:
                if noise_samples is None:
                    sig_s = np.ones(S)
                else:
                    sig_s = np.asarray(noise_samples).reshape(-1)[:S]

                # Expected NLL
                nll_draws = np.array([
                    gaussian_nll_pointwise(y_test, y_hats[i], sig_s[i]).mean()
                    for i in range(S)
                ])
                expected_nll = nll_draws.mean()

                # Predictive (mixture) NLL
                loglik = -np.stack([
                    gaussian_nll_pointwise(y_test, y_hats[i], sig_s[i])
                    for i in range(S)
                ], axis=0)  # (S, n_eval)
                lppd = (_logsumexp(loglik, axis=0) - np.log(S)).mean()
                predictive_nll = -lppd

                out_pm["expected_nll"] = expected_nll
                out_pm["predictive_nll"] = predictive_nll

                # Optional: predictive_nll on original scale (only if you also rescale sigma)
                # If your sigma posterior is on standardized scale, original sigma = sig_s * y_std.
                out_pm["predictive_nll_orig"] = predictive_nll + np.log(y_std)  # see note below

            posterior_means.append(out_pm)

            # for i in range(S):
            #     row = {
            #         'seed': seed,
            #         'N': N,
            #         'sigma': sigma,
            #         'model': model,
            #         'sparsity': sparsity,
            #         'n_eval': y_test.shape[0],
            #         'rmse': rmses[i],
            #         'rmse_orig': rmses[i] * y_std
            #     }
            #     if compute_nll:
            #        row["nll"] = gaussian_nll_pointwise(y_test, y_hats[i], sig_s[i]).mean()
            #    results.append(row)

#    return pd.DataFrame(results), pd.DataFrame(posterior_means)
    return pd.DataFrame(posterior_means)


In [ ]:
from utils.sparsity import forward_pass_relu, forward_pass_tanh, local_prune_weights

def build_masks_cache_for_all(
    all_fits,
    dataset_keys,
    models,
    sparsity_levels,
    prune_W2=False,
    method="Eabs_stability"
):
    masks_cache = {}
    for dataset_key in dataset_keys:
        for model in models:
            try:
                masks_cache[(dataset_key, model)] = precompute_global_masks(
                    all_fits=all_fits,
                    dataset_key=dataset_key,
                    model=model,
                    sparsity_levels=sparsity_levels,
                    prune_W2=prune_W2,
                    method=method
                )
            except KeyError:
                print(f"[SKIP MASKS] Missing fit for {dataset_key} -> {model}")
    return masks_cache


N100_seeds_corr = [1, 2, 3, 4, 5]
N200_seeds_corr = [6, 7, 8, 9, 10]
N500_seeds_corr = [11, 12, 13, 14, 15]
all_seeds_corr = [N100_seeds_corr, N200_seeds_corr, N500_seeds_corr]
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


def get_N_sigma_correlated(seed, all_seeds):
    if seed in all_seeds[0]:
        N=100
    elif seed in all_seeds[1]:
        N=200
    else:
        N=500
    sigma=1.00
    return N, sigma
dataset_keys_corr = []
for seed in seeds:
    N, sigma = get_N_sigma_correlated(seed, all_seeds_corr)
    dataset_keys_corr.append(f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}')

# Precompute masks once
masks_tanh_corr = build_masks_cache_for_all(tanh_fits_corr, dataset_keys_corr, model_names_tanh_corr, sparsity_levels, prune_W2=False)


In [ ]:
df_post_tanh_corr = {}
for q in sparsity_levels:
    df_post_tanh_corr[q] = compute_sparse_metrics_results_globalmask_large_eval(
        seeds=seeds,
        all_seeds=all_seeds_corr,
        models=model_names_tanh_corr,
        all_fits=tanh_fits_corr,
        get_N_sigma=get_N_sigma_correlated,
        forward_pass=forward_pass_tanh,
        folder="friedman_correlated",
        sparsity=q,
        masks_cache=masks_tanh_corr,
        prune_W2=False,
        compute_nll=True,
        noise_var_name="sigma",
        n_eval=500,
        D=10,
        standardize_y=True,
        gen_uncorr=generate_Friedman_data_v2,
        gen_corr=generate_correlated_Friedman_data_v2,
    )

In [ ]:
dataset_keys_nodewise_corr = []
for seed in seeds:
    N, sigma = get_N_sigma_correlated(seed, all_seeds_corr)
    dataset_keys_nodewise_corr.append(f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}')

# Precompute masks once
masks_tanh_nodewise_corr = build_masks_cache_for_all(tanh_fits_nodewise_corr, dataset_keys_nodewise_corr, model_names_tanh_nodewise_corr, sparsity_levels, prune_W2=False)

In [ ]:
df_post_tanh_nodewise_corr = {}
for q in sparsity_levels:
    df_post_tanh_nodewise_corr[q] = compute_sparse_metrics_results_globalmask_large_eval(
        seeds=seeds,
        all_seeds=all_seeds_corr,
        models=model_names_tanh_nodewise_corr,
        all_fits=tanh_fits_nodewise_corr,
        get_N_sigma=get_N_sigma_correlated,
        forward_pass=forward_pass_tanh,
        folder="friedman_correlated",
        sparsity=q,
        masks_cache=masks_tanh_nodewise_corr,
        prune_W2=False,
        compute_nll=True,
        noise_var_name="sigma",
        n_eval=500,
        D=10,
        standardize_y=True,
        gen_uncorr=generate_Friedman_data_v2,
        gen_corr=generate_correlated_Friedman_data_v2,
    )


In [12]:
df_post_tanh_full_corr = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_post_tanh_corr.items()],
    ignore_index=True
)

df_post_tanh_full_nodewise_corr = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_post_tanh_nodewise_corr.items()],
    ignore_index=True
)

df_all = pd.concat([df_post_tanh_full_corr, df_post_tanh_full_nodewise_corr])

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# -----------------------------
# USER SETTINGS
# -----------------------------
colors = {
    "Gaussian tanh": "C0",
    "Regularized Horseshoe tanh": "C1",
    "Dirichlet Horseshoe tanh": "C2",
    "Dirichlet Horseshoe tanh nodewise": "C2",
    "Dirichlet Student T tanh": "C3",
    "Dirichlet Student T tanh nodewise": "C3",
    "Beta Horseshoe tanh": "C4",
    "Beta Horseshoe tanh nodewise": "C4",
    "Beta Student T tanh": "C5",
    "Beta Student T tanh nodewise": "C5",
}

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]  # , 0.9
rmse_col = "posterior_mean_rmse_orig"

Ns = [100, 200, 500]

# Provide your dataframes here
dfs = [(df_all, "Uncorrelated")]#, (df_all_corr, "Correlated")]

# Models to skip (same logic as your example; edit freely)
SKIP_MODELS = {
    "Gaussian tanh",
    "Dirichlet Student T tanh",
    "Dirichlet Student T tanh nodewise",
    "Beta Student T tanh",
    "Beta Student T tanh nodewise",
}

# Aggregation across seeds:
AGG_FUN = "mean"   # "mean" or "median"
# CI / band options (optional)
SHOW_BAND = True   # <- set to False to turn off uncertainty bands
BAND_KIND = "t_ci" # "t_ci" (mean CI via t) or "quantile" (e.g., 0.25-0.75)
CI_LEVEL = 0.80    # used only for t_ci (e.g., 0.80 or 0.95)
Q_LO, Q_HI = 0.25, 0.75  # used only for quantile band


# -----------------------------
# HELPERS
# -----------------------------
def _t_crit_approx(df: int, ci_level: float) -> float:
    """
    Approximate t critical value without SciPy.
    Uses a small lookup table + normal approx fallback.
    Good enough for plotting bands.
    """
    # two-sided alpha
    alpha = 1.0 - ci_level

    # Common t critical values for two-sided intervals:
    # values correspond to ci_level in {0.80, 0.90, 0.95, 0.99} approximately
    table = {
        0.80: {1: 3.078, 2: 1.886, 3: 1.638, 4: 1.533, 5: 1.476, 6: 1.440, 7: 1.415, 8: 1.397, 9: 1.383,
               10: 1.372, 15: 1.341, 20: 1.325, 30: 1.310, 60: 1.296},
        0.90: {1: 6.314, 2: 2.920, 3: 2.353, 4: 2.132, 5: 2.015, 6: 1.943, 7: 1.895, 8: 1.860, 9: 1.833,
               10: 1.812, 15: 1.753, 20: 1.725, 30: 1.697, 60: 1.671},
        0.95: {1: 12.706, 2: 4.303, 3: 3.182, 4: 2.776, 5: 2.571, 6: 2.447, 7: 2.365, 8: 2.306, 9: 2.262,
               10: 2.228, 15: 2.131, 20: 2.086, 30: 2.042, 60: 2.000},
        0.99: {1: 63.657, 2: 9.925, 3: 5.841, 4: 4.604, 5: 4.032, 6: 3.707, 7: 3.499, 8: 3.355, 9: 3.250,
               10: 3.169, 15: 2.947, 20: 2.845, 30: 2.750, 60: 2.660},
    }

    # Snap ci_level to the nearest supported if close
    supported = np.array(sorted(table.keys()))
    nearest = supported[np.argmin(np.abs(supported - ci_level))]
    if abs(nearest - ci_level) < 1e-9:
        ci_level_use = float(nearest)
        row = table[ci_level_use]
        # pick nearest df key in that row
        df_keys = np.array(sorted(row.keys()))
        df_key = int(df_keys[np.argmin(np.abs(df_keys - df))])
        return float(row[df_key])

    # Fallback: normal approx for large df (very close to t anyway)
    # For common ci_levels this is fine; for unusual ci_level, keep it simple.
    # Two-sided z approx using an inverse-erf approximation would be overkill here;
    # use a small mapping around typical levels:
    z_map = {0.80: 1.282, 0.90: 1.645, 0.95: 1.960, 0.99: 2.576}
    ci_level_use = float(supported[np.argmin(np.abs(supported - ci_level))])
    return float(z_map[ci_level_use])


def aggregate_over_seeds(df: pd.DataFrame, metric_col: str, agg_fun: str = "mean") -> pd.DataFrame:
    """
    Aggregate metric_col across seeds within each (N, sparsity, model, sigma).
    Returns a dataframe with columns:
      N, sparsity, model, sigma, n_seeds, center, lo, hi, sd, se
    where lo/hi depend on SHOW_BAND + BAND_KIND.
    """
    group_cols = ["N", "sparsity", "model"]
    if "sigma" in df.columns:
        group_cols = ["sigma"] + group_cols

    g = df.groupby(group_cols)[metric_col]

    out = g.agg(
        n_seeds="count",
        mean="mean",
        median="median",
        sd="std",
    ).reset_index()

    out["sd"] = out["sd"].fillna(0.0)
    out["se"] = np.where(out["n_seeds"] > 0, out["sd"] / np.sqrt(out["n_seeds"]), np.nan)

    if agg_fun == "median":
        out["center"] = out["median"]
    else:
        out["center"] = out["mean"]

    # Bands
    out["lo"] = np.nan
    out["hi"] = np.nan

    if SHOW_BAND:
        if BAND_KIND == "quantile":
            q = df.groupby(group_cols)[metric_col].quantile([Q_LO, Q_HI]).unstack(-1).reset_index()
            q = q.rename(columns={Q_LO: "lo", Q_HI: "hi"})
            out = out.merge(q[group_cols + ["lo", "hi"]], on=group_cols, how="left", suffixes=("", "_q"))
            out["lo"] = out["lo_q"]
            out["hi"] = out["hi_q"]
            out = out.drop(columns=["lo_q", "hi_q"])
        elif BAND_KIND == "t_ci":
            # t-based CI for the mean: center=mean, lo/hi = mean ± t * se
            # if using median as center, still show CI around mean is weird -> use mean for CI in that case
            # so: always compute CI around mean
            def row_ci(row):
                n = int(row["n_seeds"])
                if n <= 1:
                    return (np.nan, np.nan)
                tcrit = _t_crit_approx(df=n - 1, ci_level=CI_LEVEL)
                lo = row["mean"] - tcrit * row["se"]
                hi = row["mean"] + tcrit * row["se"]
                return (lo, hi)

            lohi = out.apply(row_ci, axis=1, result_type="expand")
            out["lo"] = lohi[0]
            out["hi"] = lohi[1]
        else:
            raise ValueError("BAND_KIND must be 't_ci' or 'quantile'")

    return out


# -----------------------------
# PLOT
# -----------------------------
fig, axes = plt.subplots(2, 3, figsize=(14, 7), sharex=True, sharey=True)

for r, (df_src, row_title) in enumerate(dfs):
    # Filter upfront
    df_src = df_src.copy()
    df_src = df_src[df_src["sparsity"].isin(sparsity_levels)]
    df_src["sparsity"] = pd.Categorical(df_src["sparsity"], categories=sparsity_levels, ordered=True)

    # Aggregate across seeds
    df_agg = aggregate_over_seeds(df_src, metric_col=rmse_col, agg_fun=AGG_FUN)

    for c, N in enumerate(Ns):
        ax = axes[r, c]
        dN = df_agg[df_agg["N"] == N].copy()
        dN["sparsity"] = pd.Categorical(dN["sparsity"], categories=sparsity_levels, ordered=True)

        for model, g in dN.groupby("model", sort=False):
            if model in SKIP_MODELS:
                continue
            if model not in colors:
                # fallback if a new model label appears
                colors[model] = "C7"

            g = g.sort_values("sparsity")

            lw, alpha, ls, mk = (1.3, 0.7, "--", "v") if "nodewise" in model else (2.2, 1.0, "-", "o")

            # main line
            ax.plot(
                g["sparsity"].astype(float),
                g["center"],
                lw=lw,
                alpha=alpha,
                ls=ls,
                marker=mk,
                color=colors[model],
                label=model,
            )

            # optional band (only where we have lo/hi)
            if SHOW_BAND:
                m = g["lo"].notna() & g["hi"].notna()
                if m.any():
                    ax.fill_between(
                        g.loc[m, "sparsity"].astype(float),
                        g.loc[m, "lo"],
                        g.loc[m, "hi"],
                        alpha=0.15 if "nodewise" not in model else 0.10,
                        color=colors[model],
                        linewidth=0,
                    )

        ax.set_title(f"N={N}" if r == 0 else "")
        ax.grid(True, linestyle="--", linewidth=0.5)
        ax.set_xticks(sparsity_levels)

        if c == 0:
            ax.set_ylabel(f"{row_title}\nRMSE")
        if r == 1:
            ax.set_xlabel("sparsity")

# --- one legend for the whole figure (deduped) ---
handles, labels = axes[0, 0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))

# Put legends where you like (your original had two legends)
axes[1, 1].legend(
    by_label.values(),
    by_label.keys(),
    loc="upper left",
    frameon=False,
)
axes[0, 1].legend(
    by_label.values(),
    by_label.keys(),
    loc="upper left",
    frameon=False,
)

# Optional: annotate what the band means
if SHOW_BAND:
    if BAND_KIND == "t_ci":
        band_txt = f"{int(CI_LEVEL*100)}% t-CI (over seeds)"
    else:
        band_txt = f"quantile band [{Q_LO:.2f}, {Q_HI:.2f}] (over seeds)"
    fig.suptitle(f"Aggregation: {AGG_FUN} over seeds; Band: {band_txt}", y=1.02, fontsize=11)

plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()


In [ ]:
test = df_agg[df_agg['N']==100]
test[test['sparsity']==0.0].sort_values("mean")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# -----------------------------
# USER SETTINGS
# -----------------------------
colors = {
    "Gaussian tanh": "C0",
    "Regularized Horseshoe tanh": "C1",
    "Dirichlet Horseshoe tanh": "C2",
    "Dirichlet Horseshoe tanh nodewise": "C2",
    "Dirichlet Student T tanh": "C3",
    "Dirichlet Student T tanh nodewise": "C3",
    "Beta Horseshoe tanh": "C4",
    "Beta Horseshoe tanh nodewise": "C4",
    "Beta Student T tanh": "C5",
    "Beta Student T tanh nodewise": "C5",
}

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]  # , 0.9
rmse_col = "posterior_mean_rmse_orig"

Ns = [100, 200, 500]

# Provide your dataframes here
dfs = [(df_all, "Uncorrelated")]#, (df_all_corr, "Correlated")]

# Models to skip (same logic as your example; edit freely)
# Models to skip (same logic as your example; edit freely)
SKIP_MODELS = {
    "Gaussian tanh",
    "Dirichlet Horseshoe tanh",
    "Dirichlet Horseshoe tanh nodewise",
    "Beta Horseshoe tanh",
    "Beta Horseshoe tanh nodewise",
}

# -----------------------------
# PLOT
# -----------------------------
fig, axes = plt.subplots(2, 3, figsize=(14, 7), sharex=True, sharey=True)

for r, (df_src, row_title) in enumerate(dfs):
    # Filter upfront
    df_src = df_src.copy()
    df_src = df_src[df_src["sparsity"].isin(sparsity_levels)]
    df_src["sparsity"] = pd.Categorical(df_src["sparsity"], categories=sparsity_levels, ordered=True)

    # Aggregate across seeds
    df_agg = aggregate_over_seeds(df_src, metric_col=rmse_col, agg_fun=AGG_FUN)

    for c, N in enumerate(Ns):
        ax = axes[r, c]
        dN = df_agg[df_agg["N"] == N].copy()
        dN["sparsity"] = pd.Categorical(dN["sparsity"], categories=sparsity_levels, ordered=True)

        for model, g in dN.groupby("model", sort=False):
            if model in SKIP_MODELS:
                continue
            if model not in colors:
                # fallback if a new model label appears
                colors[model] = "C7"

            g = g.sort_values("sparsity")

            lw, alpha, ls, mk = (1.3, 0.7, "--", "v") if "nodewise" in model else (2.2, 1.0, "-", "o")

            # main line
            ax.plot(
                g["sparsity"].astype(float),
                g["center"],
                lw=lw,
                alpha=alpha,
                ls=ls,
                marker=mk,
                color=colors[model],
                label=model,
            )

            # optional band (only where we have lo/hi)
            if SHOW_BAND:
                m = g["lo"].notna() & g["hi"].notna()
                if m.any():
                    ax.fill_between(
                        g.loc[m, "sparsity"].astype(float),
                        g.loc[m, "lo"],
                        g.loc[m, "hi"],
                        alpha=0.15 if "nodewise" not in model else 0.10,
                        color=colors[model],
                        linewidth=0,
                    )

        ax.set_title(f"N={N}" if r == 0 else "")
        ax.grid(True, linestyle="--", linewidth=0.5)
        ax.set_xticks(sparsity_levels)

        if c == 0:
            ax.set_ylabel(f"{row_title}\nRMSE")
        if r == 1:
            ax.set_xlabel("sparsity")

# --- one legend for the whole figure (deduped) ---
handles, labels = axes[0, 0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))

# Put legends where you like (your original had two legends)
axes[1, 1].legend(
    by_label.values(),
    by_label.keys(),
    loc="upper left",
    frameon=False,
)
axes[0, 1].legend(
    by_label.values(),
    by_label.keys(),
    loc="upper left",
    frameon=False,
)

# Optional: annotate what the band means
if SHOW_BAND:
    if BAND_KIND == "t_ci":
        band_txt = f"{int(CI_LEVEL*100)}% t-CI (over seeds)"
    else:
        band_txt = f"quantile band [{Q_LO:.2f}, {Q_HI:.2f}] (over seeds)"
    fig.suptitle(f"Aggregation: {AGG_FUN} over seeds; Band: {band_txt}", y=1.02, fontsize=11)

plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()
